In [4]:
# Setup the Jupyter version of Dash
from dash import Dash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
#  CRUD Python module name
DB = 'AAC'
COL = 'animals'

# Connect to database via CRUD Module
db = AnimalShelter(DB, COL)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
#print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = Dash(__name__)

#Add in Grazioso Salvare’s logo
image_filename = 'Grazioso_Salvare_Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#Place the HTML image tag in the line below into the app.layout code according to your design
#Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(html.H3("Pushpa Laxman SNHU")),
    html.Center(html.A([html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),style={'width': '300px', 'height': '200px'} )],href="https://www.snhu.edu")),
    html.Hr(),
    html.Div(
        
#Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    ),
    html.Hr(),
    dcc.RadioItems(id='filter-type', options=[
       {'label': 'Water Rescue', 'value': 'Water_Rescue'},
       {'label': 'Mountain Rescue', 'value': 'Mountain_Rescue'},       
       {'label': 'Disaster Rescue', 'value': 'Disaster_Rescue'},
       {'label': 'Reset', 'value': '{}'},

   ],inline=True),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         #Set up the features for your interactive data table to make it user-friendly for your client
                         editable = False,
                         filter_action = "native",
                         sort_action = "native",
                         sort_mode = "multi",
                         column_selectable = False,
                         row_selectable = "single",
                         row_deletable = False,
                         selected_columns = [],
                         selected_rows = [0],
                         page_action = "native",
                         page_current = 0,
                         page_size = 10

                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',
            style={'width': '50%'}  # set the width to half the page size

            ),
            # add the bar chart
        html.Div(
            id='bar-id', # bar chart for outcome type by count
            className='col s12 m6',
            style={'width': '50%'}  # set the width to half the page size
        )
        
        
        ]),
    # below, set up the pie and bar chart side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[html.Div(
            id='map-id',
            className='col s12 m6',
            ),
        
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),Output('datatable-id','columns'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    #print(filter_type)
    # filter_type is Water Rescue
    if filter_type == 'Water_Rescue':
        df = pd.DataFrame.from_records(db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix","Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
            }))
    # filter_type is  Mountain Rescue
    elif filter_type == 'Mountain_Rescue':
        df = pd.DataFrame.from_records(db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepard","Alaskan Malamute","Old English Sheepdog","Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":26.0, "$lte":156.0}
       }))
    # filter_type is Disaster Rescue
    elif filter_type == 'Disaster_Rescue':
        df = pd.DataFrame.from_records(db.read({
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher","German Shepard","Golden Retriever", "Bloodhound","Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte":20.0, "$lte":300.0}
       }))
    # filter_type is Reset
    else:
        df = pd.DataFrame.from_records(db.read({}))
    df.drop(columns=['_id'],inplace=True)
    #print(len(df.to_dict(orient='records')))
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    
    return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    if viewData is None:
        return
    df = pd.DataFrame.from_records(viewData)
    figure = px.pie(df, names = 'breed', title = 'Preferred Animals')
    figure.update_traces(textposition = 'inside', textinfo = 'percent+label')
    return [
        dcc.Graph(            
          figure = figure
        )    
    ]

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('bar-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('filter-type', 'value')]
)
def update_bar(viewData, filterValue, **kwargs):
    # Create a data frame from the data
    dff = pd.DataFrame(viewData)
    # if outcome type is in the data
    if 'outcome_type' in dff.columns:
        dff = dff['outcome_type'].value_counts().reset_index()
        # rename the columns and define the df
        dff.columns = ['Outcome Type', 'Count']
        # Create a bar chart
        bar_fig = px.bar(
            dff,
            x='Outcome Type',      # X-axis variable
            y='Count',             # Y-axis variable
            color='Outcome Type',  # Color of the bars 
            title='Preferred Animals by Outcome Type',  # Title the chart
            template='plotly_white'  # Set the template style
        )
        # set labels to diagonal 
        bar_fig.update_layout(xaxis_tickangle=45)

        # Return the graph
        return [dcc.Graph(figure=bar_fig)]
    # if age upon outcome is not in the data return an empty graph
    else:
        empty_fig = px.bar(title='No Data Available', template='plotly_white')
        return [dcc.Graph(figure=empty_fig)]


#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run(debug=True)
